In [3]:
# tsv 형식의 파일
import glob
import os
import io
import string

# 훈련 데이터의 tsv 파일 작성
f = open('./data/text_train.tsv', 'w')

path =

전처리 및 단어 분할 함수 정의

In [7]:
import string
import re

# 다음 기호는 스페이스(공백)로 치환(쉼표, 마침표 제외)
# punctuation은 구두점
print("구두점 문자:", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 전처리
def preprocessing_text(text):
    # 개행 코드 삭제
    text = re.sub('<br />', '', text)
    
    # 쉼표, 마침표 이외의 기호를 공백으로 치환
    for p in string.punctuation:
        if (p == '.') or (p == ','):
            continue
        else:
            text = text.replace(p, ' ')
    
    # 쉼표, 마침표의 전후에 공백 추가
    text = text.replace('.', ' . ')
    text = text.replace(',', ' , ')
    return text

# 띄어쓰기(이번에는 영어 데이터 이며 임시로 공백으로 구분)
def tokenizer_punctuation(text):
    return text.strip().split()

# 전처리 및 띄어쓰기를 포함한 함수 정의
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret

print(tokenizer_with_preprocessing('I like cats.'))

구두점 문자: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cats', '.']


Transformer 구현(분류 작업용)

In [ ]:
# Embedder 모듈
import torch.nn as nn

class Embedder(nn.Module):
    '''id로 표시된 단어를 벡터로 변환'''
    
    def __init__(self, text_embedding_vectors):
        super(Embedder, self).__init__()
        
        self.embeddings = nn.Embedding.from_pretrained(
            embeddings=text_embedding_vectors, freeze=True)
        # freeze=True에 의해 역전파로 갱신되지 않고 변하지 않는다
        
    def forward(self, x):
        x_vec = self.embeddings(x)
        
        return x_vec

훈련 및 검증 함수의 구현과 실행

In [ ]:
# 모델을 학습시키는 함수 작성
import torch
import torch.nn as nn
import torch.optim as optim

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    # GPU 사용 확인
    device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
    print('사용장치: ', device)
    print('-----start-----')
    
    # 네트워크를 MPS로
    net.to(device)
    
    # 네트워크가 어느 정도 고정되면 고속화 시킨다
    torch.backends.cudnn.benchmark = True
    
    # 에폭 루프
    for epoch in range(num_epochs):
        # 에폭별 훈련 및 검증 루프
        for phase in ['train','val']:
            if phase == 'train':
                net.train() # 모델을 훈련 모드로
            else:
                net.eval() # 모델을 검증 모드로
            epoch_loss = 0.0 # 에폭 손실의 합
            epoch_corrects = 0 # 에폭의 정답 수
            
            # 데이터 로더에서 미니 배치를 꺼내는 루프
            for batch in (dataloaders_dict[phase]):
                # 배치는 텍스트와 라벨의 사전 오브젝트
                
                # GPU를 사용할 수 있으면 GPU로 데이터를 보낸다
                inputs = batch.Text[0].to(device)
                labels = batch.Label.to(device)
                
                # 옵티마이저 초기화
                optimizer.zero_grad()
                
                # 순전파 계산
                # False 시(추론 모드) 역전파나 가중치 업데이트가 일어나지 않음
                with torch.set_grad_enabled(phase == 'train'):
                     
                    # 마스크 작성
                    input_pad = 1 # 단어 ID에 있어서 '<pad>': 1 이므로
                    input_mask = (inputs != input_pad)
                    
                    # Transformer에 입력
                    outputs, _, _ = net(inputs, input_mask)
                    loss = criterion(outputs, labels) # 손실 계산
                    
                    _, preds = torch.max(outputs, 1) # 라벨 예측
                    
                    # 훈련 시에는 역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    # 결과 계산
                    epoch_loss += loss.item() * inputs.size(0) # 손실 합계 갱신
                    # 정답 수 합계를 갱신
                    epoch_corrects += torch.sum(preds == labels.data)
                    
                # 에폭별 손실과 정답률
                epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
                epoch_acc = epoch_corrects.double(
                ) / len(dataloaders_dict[phase].dataset)
                
                print('Epoch {}/{} | {:^5} | Loss: {:.4f} Acc: {:.4f}'.
                     format(epoch+1, num_epochs, phase, epoch_loss, epoch_acc))
                
            return net
        
# 손실 함수 설정
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax() 를 계산한 뒤 nn.NLLLoss(negative log likelihood loss) 계산

# 최적화 기법 설정
learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# 학습 및 검증 실행
num_epochs = 10
net_trained = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

테스트 데이터에서의 추론과 판정 근거의 시각화

In [ ]:
import torch
# device
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

net_trained.eval() # 모델을 검증 모드로
net_trained.to(device)

epoch_corrects = 0 # 에폭의 정답 수

for batch in (test_dl): # 테스트 데이터의 데이터 로더
    # 배치는 텍스트와 라벨의 사전 오브젝트
    
    # MPS를 사용할 수 있다면 MPS로 데이터를 보낸다
    inputs = batch.Text[0].to(device)
    labels = batch.Label.to(device)
    
    # 순전파 계산
    with torch.set_grad_enabled(False):
        
        # 마스크 작성
        input_pad = 1 # 단어 ID에 있어서 '<pad>':1 이므로
        input_mask = (inputs != input_pad)
        
        # Transformer에 입력
        outputs, _, _ = net_trained(inputs, input_mask)
        _, preds = torch.max(outputs, 1) # 라벨 예측
        
        # 결과 계산
        # 정답 수 합계 갱신
        epoch_corrects += torch.sum(preds == labels.data)
        
# 정답률
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('테스트 데이터 {}개의 정답률: {:.4f}'.format(len(test_dl.dataset), epoch_acc))